In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
# from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Avoid repeated and unnecessary importations of the dataset
# from fCC's Static Assets.
df = dataset.copy()
df.head()

In [ ]:
# Any necessity to impute any value?
df.isna()\
  .sum()

In [ ]:
# One-hot encode categorical values to be parsed as numerical ones
categoricals = ['sex',
                'smoker',
                'region']
df = pd.get_dummies(dataset,
                    columns = categoricals,
                    drop_first = True)

df.head()

In [ ]:
# For the train dataset, create a random sample which contains 80% of the observations
train_dataset = df.sample(frac = .8,
                          random_state = 0)

# Assign the test dataset
test_dataset = df.drop(train_dataset.index)

# Pop off the 'expenses' colmuns
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')


In [ ]:
# Add output layers
input_shape = len(train_dataset.keys())

model = keras.Sequential(
    [
    layers.Dense(128,
                 activation = 'relu',
                 input_shape = [input_shape]),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(32, activation = 'relu'),
    layers.Dense(16, activation = 'relu'),
    layers.Dense(1, activation = 'relu')
  ]
    )

# Compile the model:
# Implement an 'Adaptive Moment Estimation' algorithm
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam

optimizer = tf.keras.optimizers.Adam(
    learning_rate = .008
)

model.compile(loss = 'mae',
              optimizer = optimizer,
              metrics = ['mae','mse'])

model.summary()


In [ ]:

# Trian the model
# https://keras.io/api/callbacks/model_checkpoint/:

EPOCHS = 500
checkpoint_filepath = './tmp/ckpt/fcchealthcosts.model.keras'

checkpoint_callback  = [tf.keras.callbacks\
                        .ModelCheckpoint(filepath = checkpoint_filepath,
                                         monitor = 'val_loss',
                                         verbose = 1,
                                         save_best_only = True,
                                         mode = 'min'
                                         )
]
# https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History:
# https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit:

history = model.fit(train_dataset,
                    train_labels,
                    epochs = EPOCHS,
                    verbose = 1,
                    validation_split = .2,
                    shuffle = False,
                    callbacks = checkpoint_callback
)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)


In [ ]:
# Compute some descriptive statistics of the test dataset
test_descript_stats = test_dataset.describe()\
                           .transpose()\
                           .round(3)
test_descript_stats